In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from skimage.transform import resize

Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename):
    
    # todo
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)
    if ds.Modality != 'DX':
        return None
    if ds.PatientPosition != 'PA':
        if ds.PatientPosition != 'AP':
            return None
    if ds.BodyPartExamined != 'CHEST':
        return None
    image = np.stack(ds.PixelData)
    img = image.astype(np.int16)
    return img/255.0
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    proc_img = (img - img_mean) / img_std
    resized_img = resize(proc_img, img_size, preserve_range=True, anti_aliasing=False)
    print('image shape: {}'.format(resized_img.shape))
    #a = np.asarray(proc_img)
    #image = Image.fromarray(a, mode = 'P')
    #proc_img = image.resize(img_size) 
    img = resized_img.reshape(1,224,224,3)
    return img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = keras.models.model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights(weight_path)
    print("Loaded model from disk")
 
    # evaluate loaded model on test data
    optimizer = keras.optimizers.Adam(lr=1e-4)
    loss = 'binary_crossentropy'
    metrics = ['binary_accuracy']
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)    
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    y_pred = model.predict(img)
    prediction = (y_pred > thresh)   
    
    return prediction 

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json'
weight_path = 'xray_class_my_model.best.hdf5'

IMG_SIZE=(224,224,3) # This might be different if you did not use vgg16
img_mean = 0.492
img_std = 0.244

my_model = load_model(model_path, weight_path)
thresh = 0.55 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print('is pneumonia positive: {}'.format(pred))

Loaded model from disk
Load file test1.dcm ...
image shape: (224, 224, 3)
is pneumonia positive: [[ True]]
Load file test2.dcm ...
image shape: (224, 224, 3)
is pneumonia positive: [[ True]]
Load file test3.dcm ...
image shape: (224, 224, 3)
is pneumonia positive: [[ True]]
Load file test4.dcm ...
Load file test5.dcm ...
Load file test6.dcm ...
